In [4]:
import joblib

In [7]:

# Specify the path to the file

path = 'trained_models/'
version = path + '19_05_2024__20-31'

dataset = 'credit_score'

# Load the models

for model in ['rf', 'rbf_svm', 'dnn']:
    
    # Load the model
    model = joblib.load(version + '/' + dataset + '_' + model + '.joblib')